### Assignment 7 - Data


In [1]:
### Set up connection to WRDS...

import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import wrds
db=wrds.Connection(wrds_username='perusset')
db.create_pgpass_file()

C:\Users\perusset\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Loading library list...
Done
Enter your WRDS username [perusset]:perusset
Enter your password:········


In [ ]:

### Download and format data

#---------------------------------------------
# Risk Free Rate 
#---------------------------------------------
Rf=db.raw_sql("""select  mcaldt,tmytm 
           from crsp.tfz_mth_rf            
            where kytreasnox = 2000001 
           and mcaldt>='1980-01-01'
            and mcaldt<='2023-12-31'""", date_cols=['mcaldt'])
Rf['tmytm']=Rf['tmytm']/12/100
Rf=Rf.rename(columns={ "mcaldt": "date","tmytm": "rf"})

#---------------------------------------------
# Value Weighted Index Returns
#---------------------------------------------
Rm=db.raw_sql("""select  date,vwretd from crsp.msi 
                where date>='1980-01-01' and date<='2023-12-31'
                """,date_cols=['date'])
Rm = Rm.rename(columns={'vwretd':'Rm'})

#---------------------------------------------
# Stock Returns 
#---------------------------------------------

stock_data = db.raw_sql("""
                    select a.permno, a.date, a.ret, a.shrout, a.prc, b.siccd 
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1980' and '12/31/2023'
                      """, date_cols=['date']) 
stock_data = stock_data.rename(columns={'ret':'Rn'})
# Restrict to SP500 
SP500_constituents = db.get_table(library='crsp', table='msp500list')
sp500 = pd.merge(SP500_constituents, stock_data,on=['permno'], how='left')
sp500['ending'] = sp500['ending'].fillna(sp500.date.max())
sp500 = sp500[(sp500.date >= sp500.start) & (sp500.date <= sp500.ending)].copy()

In [80]:

#---------------------------------------------
# Industry Definitions  and Returns 
#---------------------------------------------
Industry_Returns = pd.read_csv('Data/12_Industry_Portfolios.txt', delimiter= '\s+')
Industry_Definitions = pd.read_csv('Data/48_Industries.csv')
Industry_Definitions = Industry_Definitions.rename(columns={'SIC':'siccd'})
Industries = Industry_Returns.columns.tolist()
Industry_Names = pd.DataFrame(Industries,index=np.arange(1,13)).reset_index()
Industry_Names.columns=['Industry','Name']
Industry_Returns = Industry_Returns.reset_index().rename(columns = {'index':'dateM'})
Industry_Returns['dateM'] = pd.to_datetime(Industry_Returns['dateM'], format='%Y%m', errors='coerce')+pd.offsets.MonthEnd(0)


In [81]:
Industry_Returns.head()

,dateM,NoDur,Durbl,Manuf,Enrgy,Chems,BusEq,Telcm,Utils,Shops,Hlth,Money,Other
0,1926-07-31,1.45,15.55,3.67,-1.18,8.01,3.16,0.83,7.04,0.11,1.77,-0.02,2.22
1,1926-08-31,3.97,3.68,2.42,3.47,5.14,1.97,2.17,-1.69,-0.71,4.25,4.47,4.34
2,1926-09-30,1.14,4.80,-0.07,-3.39,5.30,-0.34,2.41,2.04,0.21,0.69,-1.61,0.37
3,1926-10-31,-1.24,-8.23,-3.16,-0.78,-4.55,-5.38,-0.11,-2.63,-2.29,-0.57,-5.51,-2.73
4,1926-11-30,5.20,-0.19,3.82,0.01,5.11,4.79,1.63,3.71,6.43,5.42,2.34,2.10


In [90]:
### Merge different data-sources

#---------------------------------------------
# Merge Data 
#---------------------------------------------
data = pd.merge(sp500, Rf, on=['date'], how='left')
data = pd.merge(data, Rm, on=['date'], how='left')
data = pd.merge(data, Industry_Definitions[['siccd','FF_12']],on=['siccd'], how='left')
#data['dateM'] = data['date'].astype('datetime64[M]')
data['dateM'] = data['date']

data = pd.merge(data, Industry_Returns,on=['dateM'], how='left')
rf_array = np.array(data['rf'])

data[Industries] = data[Industries]/100 - rf_array[:,np.newaxis]


In [91]:

#---------------------------------------------
# Define Variables and drop obs
#---------------------------------------------
data = data.sort_values(['permno','date'])
data['mcap'] = np.abs(data['prc'])*data['shrout']
data['mcap_l'] = data.groupby(['permno'])['mcap'].shift(1)
data['Rn_f'] = data.groupby(['permno'])['Rn'].shift(-1)
data['const'] = 1
data['Rn_e'] = data['Rn']-data['rf']
data['Rm_e'] = data['Rm']-data['rf']
data['w_m'] = data['mcap_l']/data.groupby(['date'])['mcap_l'].transform('sum')
data = data.dropna(subset=['mcap_l','Rn_e','Rm_e','FF_12']).copy()

In [92]:
data

,permno,start,ending,date,Rn,shrout,prc,siccd,rf,Rm,...,Hlth,Money,Other,mcap,mcap_l,Rn_f,const,Rn_e,Rm_e,w_m
1,10006,1957-03-01,1984-07-18,1980-02-29,-0.058795,8859.0,38.50000,3743.0,0.011347,0.001015,...,-0.057647,-0.061547,-0.027247,3.410715e+05,3.676485e+05,-0.172078,1,-0.070142,-0.010332,0.000491
2,10006,1957-03-01,1984-07-18,1980-03-31,-0.172078,8859.0,31.87500,3743.0,0.012542,-0.120224,...,-0.040942,-0.101542,-0.169042,2.823806e+05,3.410715e+05,-0.027451,1,-0.184620,-0.132766,0.000456
3,10006,1957-03-01,1984-07-18,1980-04-30,-0.027451,8859.0,31.00000,3743.0,0.007698,0.052287,...,0.035102,0.060602,0.044802,2.746290e+05,2.823806e+05,0.084677,1,-0.035149,0.044589,0.000421
4,10006,1957-03-01,1984-07-18,1980-05-30,0.084677,8859.0,33.00000,3743.0,0.005985,0.060091,...,NaN,NaN,NaN,2.923470e+05,2.746290e+05,-0.007576,1,0.078692,0.054106,0.000391
5,10006,1957-03-01,1984-07-18,1980-06-30,-0.007576,8889.0,32.75000,3743.0,0.005168,0.037714,...,-0.003068,0.043832,0.048332,2.911148e+05,2.923470e+05,0.091603,1,-0.012744,0.032546,0.000397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264473,93436,2020-12-21,2023-12-29,2023-08-31,-0.034962,3173994.0,258.07999,9999.0,0.004463,-0.020605,...,NaN,NaN,NaN,8.191443e+08,8.488212e+08,-0.030456,1,-0.039425,-0.025068,0.021241
264474,93436,2020-12-21,2023-12-29,2023-09-29,-0.030456,3179000.0,250.22000,9999.0,0.004472,-0.047390,...,NaN,NaN,NaN,7.954494e+08,8.191443e+08,-0.197346,1,-0.034928,-0.051862,0.020870
264475,93436,2020-12-21,2023-12-29,2023-10-31,-0.197346,3178921.0,200.84000,9999.0,0.004475,-0.028867,...,NaN,NaN,NaN,6.384545e+08,7.954494e+08,0.195379,1,-0.201821,-0.033342,0.021292
264476,93436,2020-12-21,2023-12-29,2023-11-30,0.195379,3178921.0,240.08000,9999.0,0.004455,0.092057,...,NaN,NaN,NaN,7.631954e+08,6.384545e+08,0.034988,1,0.190924,0.087602,0.017468


In [93]:
# Export data
data.to_csv('data.csv', sep=';',index=False) # export data


In [94]:
Industries

['NoDur',
 'Durbl',
 'Manuf',
 'Enrgy',
 'Chems',
 'BusEq',
 'Telcm',
 'Utils',
 'Shops',
 'Hlth',
 'Money',
 'Other']